
<div id="navegacion" prev="../Ejercicios_Tema_7/ej6.html" next=""></div>

<div class="section-title">Section 6.5</div>

#  Movilens 1M


Movilens Dataset collects data from 6,000 users and 4,000 movies. The information is
distributed in 3 tables.

- [ratings.dat](./ml-1m/ratings.dat) - The file collects user rating data. The columns are: User, MovieID, Rating, Date.

- [movies.dat](./ml-1m/movies.dat) - Movie data. The columns are: MovieID, Title, Genre
- [users.dat](./ml-1m/users.dat) - Personal data of users. The columns are: User, Gender, Age, Occupation,
ZipCode.


All this information is used very frequently in recommender systems that apply learning
algorithms. Information on the content of the files can be found in the [README](./ml-1m/README) link.


## Import data

Create a DataFrame for each of the files. Use the function `head()` to display the first data. The
ratings file has a date type field. Use the parameter `parse_dates` to properly load the dates.

In [3]:
import pandas as pd

In [4]:
# Sol:
ratings = pd.read_csv('ratings.dat', sep=':', header=None, names=['User','MovieID','Rating','Date'], parse_dates=True)
movies = pd.read_csv('movies.dat', sep=':', header=None, names=['MovieID','Title','Genre'])
users = pd.read_csv('users.dat', sep=':', header=None, names=['User','Gender','Age','Occupation','ZipCode'])


In [5]:
ratings.head(5)

,User,MovieID,Rating,Date
0,1139,377,4,2000-11-22 08:29:17
1,1524,3857,4,2000-11-20 19:57:23
2,4823,2987,3,2000-07-09 00:25:18
3,336,2105,5,2000-12-09 12:38:32
4,5131,1752,2,2000-06-27 20:32:25
5,4786,2006,4,2000-07-07 22:29:26
6,2355,300,4,2000-11-16 19:45:09
7,3018,2717,3,2000-10-02 23:20:30
8,4169,1214,4,2000-10-15 04:58:53
9,5056,2011,4,2000-07-01 23:12:44


In [6]:
movies.head(5)

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
users.head(5)

,User,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


## X-Men ratings

Users rate the movies with values from 1 to 5 where 1 is the worst score and 5 is the best. The
score is a categorical variable. Find the number of votes of each type (the frequency of votes)
for the movie entitled `X-Men (2000)`.



In [42]:
# Sol:
XMRatings = ratings[ratings.MovieID == movies[movies.Title == 'X-Men (2000)'].MovieID.values[0]]
res = XMRatings.Rating.value_counts()
print("Rating/Count")
res

Rating/Count


4    4
5    3
3    2
2    1
Name: Rating, dtype: int64

## Average score

Calculate the average score of each movie by user gender. Design a function called
`puntuacion_media` that returns a Dataframe where the index is the movie title and the columns
are 'F' and 'M' (Female and Male respectively).

__Remark__:

There are several ways to do it. Here are some suggestions:
* We need to combine the three tables since we have, on the one hand, the ratings, on the
other the title of the movie and on the other the gender of the users. The operation
`merge` will allow you to perform the combination.
    * First match the ratings with the movies by the 'id_peli' field.
    * Combine the above result with the users by the 'id_usuario' field.
    
* Once the three tables have been combined into a single table:
    * solution 1:  Group by `id_peli` and user `sexo` (genre). The aggregation function is the mean `mean()`. To get the requested table use `unstack` to undo the multi index.
    * solution 2: Check the doc for function `pivot_table`. Use the function `pivot_table`. It is
worth trying to solve it also using this option.


In [46]:
# Sol:
def puntuacion_media ():
  """Compute the average score by movie and user genre

  Example:
      >>> puntuacion_media()

      genero_u                            F          M
      titulo
      $1,000,000 Duck (1971)       3.375000    2.761905
      'Night Mother (1986)         3.388889    3.352941
      'Til There Was You (1997)    2.675676    2.733333
      ...
      ...
  """
  ratings_movies = ratings.merge(movies, on='MovieID')
  ratings_movies_users = ratings_movies.merge(users, on='User')

  group_ID_gender = ratings_movies_users.groupby(['Title', 'Gender']).Rating.mean().round(2)

  return group_ID_gender.unstack().fillna('---')

# Sol 2:
def puntuacion_media_2():
  ratings_movies = ratings.merge(movies, on='MovieID')
  ratings_movies_users = ratings_movies.merge(users, on='User')

  sol = pd.pivot_table(ratings_movies_users, values='Rating', index='Title', columns='Gender', fill_value='---')

  return  sol

In [47]:
puntuacion_media ()

Gender,F,M
Title,,
"$1,000,000 Duck (1971)",3.0,---
'Til There Was You (1997),---,1.0
"'burbs, The (1989)",1.0,2.0
...And Justice for All (1979),---,4.33
10 Things I Hate About You (1999),4.2,4.5
...,...,...
"Young Poisoner's Handbook, The (1995)",---,3.0
Young Sherlock Holmes (1985),---,3.67
Your Friends and Neighbors (1998),---,4.0


In [48]:
puntuacion_media_2()

Gender,F,M
Title,,
"$1,000,000 Duck (1971)",3.0,---
'Til There Was You (1997),---,1.0
"'burbs, The (1989)",1.0,2.0
...And Justice for All (1979),---,4.333333
10 Things I Hate About You (1999),4.2,4.5
...,...,...
"Young Poisoner's Handbook, The (1995)",---,3.0
Young Sherlock Holmes (1985),---,3.666667
Your Friends and Neighbors (1998),---,4.0
